In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv("Data_Eng_v7.csv", low_memory=False)

In [4]:
df = df.dropna()
df = df.drop_duplicates()

In [5]:
df.tail()

,Unnamed: 0.1,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,Unnamed: 0,price_per_sqm,Closest_supermarket_Distance,Closest_supermarket_postal,Closest_industrial_Distance,Closest_industrial_name,School_Density_Within_1KM,Special_School_Density_Within_1KM,Closest_rental_Distance,Closest_Mall_Distance
192061,192061,2024-10,YISHUN,5 ROOM,870,YISHUN ST 81,04 TO 06,122.0,Improved,1987,...,417,6453.0,0.311652,760507,2.935779,Seletar Aerospace Park,3,0,0.365474,0.660889
192062,192062,2024-10,YISHUN,EXECUTIVE,654,YISHUN AVE 4,10 TO 12,164.0,Apartment,1992,...,417,6453.0,0.475721,760675,2.000149,Yishun Industrial Park A,3,0,0.532268,0.572021
192063,192063,2024-10,YISHUN,EXECUTIVE,654,YISHUN AVE 4,04 TO 06,164.0,Apartment,1992,...,417,6453.0,0.475721,760675,2.000149,Yishun Industrial Park A,3,0,0.532268,0.572021
192064,192064,2024-10,YISHUN,EXECUTIVE,358,YISHUN RING RD,01 TO 03,145.0,Maisonette,1988,...,417,6453.0,0.259860,760418,1.690048,Admiralty East Industrial Estate,3,0,0.523747,0.845712
192065,192065,2024-10,YISHUN,EXECUTIVE,824,YISHUN ST 81,10 TO 12,142.0,Apartment,1987,...,417,6453.0,0.315496,760845,3.084699,Yishun Industrial Park A,2,0,0.841459,1.008147


In [6]:
# Convert 'month' to datetime and extract the year
df['month'] = pd.to_datetime(df['month'], format='%Y-%m')
df['month_year'] = df['month'].dt.year

# df = df[(df['month'] < "2021-01")]

In [7]:
df.columns

Index(['Unnamed: 0.1', 'month', 'town', 'flat_type', 'block', 'street_name',
       'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'remaining_lease', 'resale_price', 'residential', 'commercial',
       'market_hawker', 'miscellaneous', 'multistorey_carpark',
       'precinct_pavilion', 'LATITUDE', 'LONGITUDE', 'POSTAL', 'time_cbd',
       'Distance_CBD', 'Closest_mrt_Distance', 'Closest_mrt_Station',
       'busstop_Density_Within_1KM', 'Closest_hawkercentre_Distance',
       'Closest_Hawker', 'Unnamed: 0', 'price_per_sqm',
       'Closest_supermarket_Distance', 'Closest_supermarket_postal',
       'Closest_industrial_Distance', 'Closest_industrial_name',
       'School_Density_Within_1KM', 'Special_School_Density_Within_1KM',
       'Closest_rental_Distance', 'Closest_Mall_Distance', 'month_year'],
      dtype='object')

In [8]:
print(df['storey_range'].unique())
storey_list = sorted(df['storey_range'].unique(), key = lambda x: x[0:1])
print(storey_list)

storey_dict = {}
for i in range(len(storey_list)):
    storey_dict[storey_list[i]] = i

print(storey_dict)

df['storey_range'] = df['storey_range'].apply(lambda x: storey_dict.get(x))

['10 TO 12' '01 TO 03' '04 TO 06' '07 TO 09' '13 TO 15' '19 TO 21'
 '22 TO 24' '16 TO 18' '34 TO 36' '28 TO 30' '37 TO 39' '49 TO 51'
 '25 TO 27' '40 TO 42' '31 TO 33' '46 TO 48' '43 TO 45']
['01 TO 03', '04 TO 06', '07 TO 09', '10 TO 12', '13 TO 15', '19 TO 21', '16 TO 18', '22 TO 24', '28 TO 30', '25 TO 27', '34 TO 36', '37 TO 39', '31 TO 33', '49 TO 51', '40 TO 42', '46 TO 48', '43 TO 45']
{'01 TO 03': 0, '04 TO 06': 1, '07 TO 09': 2, '10 TO 12': 3, '13 TO 15': 4, '19 TO 21': 5, '16 TO 18': 6, '22 TO 24': 7, '28 TO 30': 8, '25 TO 27': 9, '34 TO 36': 10, '37 TO 39': 11, '31 TO 33': 12, '49 TO 51': 13, '40 TO 42': 14, '46 TO 48': 15, '43 TO 45': 16}


In [9]:
df.head()
df["storey_range"]

0         3
1         0
2         0
3         1
4         0
         ..
192061    1
192062    3
192063    1
192064    0
192065    3
Name: storey_range, Length: 192066, dtype: int64

In [10]:
df = df.drop(columns = ["Unnamed: 0.1", "Unnamed: 0", "LATITUDE", "LONGITUDE", "street_name", "block", "Closest_Hawker", "POSTAL","residential", "remaining_lease", "Closest_supermarket_postal", 'Closest_industrial_name', 'time_cbd','commercial','multistorey_carpark'])

In [11]:
df = pd.get_dummies(df)


In [12]:
df['property_age'] = df['month_year'] - df['lease_commence_date']
df = df[(df["month"] < "2024-09")]

In [13]:
df = df.drop(columns = ["month_year", "month", "lease_commence_date"])

In [14]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X = df.drop('resale_price', axis=1)
y = df["resale_price"]

train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

df = df.drop("resale_price", axis = 1)
feature_list = list(df.columns)
# Convert to numpy array
features = np.array(df)



In [15]:
# Import the model we are using
from math import sqrt
xgb = XGBRegressor(n_estimators = 100, random_state = 1)
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Different learning rates
    'max_depth': [3, 6, 10],                  # Depth of trees
    'n_estimators': [50, 100, 200],           # Number of boosting rounds
    'subsample': [0.7, 0.8, 1.0],             # Fraction of samples used for training
    'colsample_bytree': [0.7, 0.8, 1.0]       # Fraction of features used for training
}

grid_search = GridSearchCV(xgb, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(train_features, train_labels)
print("Best set of hyperparameters: ", grid_search.best_params_)
print("Best score: ", sqrt(-grid_search.best_score_))

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best set of hyperparameters:  {'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 0.8}
Best score:  26410.651977702524


In [16]:
from sklearn.metrics import mean_squared_error
best_model = grid_search.best_estimator_
y_pred = best_model.predict(test_features)
mse_test = mean_squared_error(test_labels, y_pred)

In [17]:
from sklearn.metrics import mean_squared_error
y_pred = best_model.predict(test_features)
rms = mean_squared_error(test_labels, y_pred, squared=False)

from sklearn.metrics import r2_score
r2 = r2_score(test_labels, y_pred)

C:\Users\ranoc\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [18]:
from math import sqrt
print(rms)

25914.390337996956


In [19]:
print(r2)

0.9783237530633263
